<center>
  
# Synthetic Data Evaluation

</center>

In this notebook, we will evaluate the generated synthetic data using the CLAVADDPM algorithm based on various metrics. The notebook is organized as follows:

1. [Imports and Setup]()


2. [Density Estimation of Single Column and Pair-wise Correlation]()
    
    
3. [$\alpha$-Precision and $\beta$-Recall ]()

    
4. [Machine Learning Efficiency]()


5. [Privacy Protection: Distance to Closest Record (DCR)]()


6. [Detection: Classifier Two Sample Tests (C2ST)]()

# Imports and Setup

First, we will import functions to evaluate the data using various metrics. Then, we will define the paths to the real train and test data, as well as the CLAVADDPM
 generated data.

Adjust the paths below to match the locations of data files on your system.

To run this notebook, a `trans_info.json` file, similar to the one generated during TabSyn model preprocessing, is required. A sample is available at `configs/info_files/trans_info.json`, where you can modify the data_path if necessary.

**To obtain more accurate results, use the updated synthetic data as mentioned in the `ClavaDDPM.ipynb`, where the floating-point values are discarded for categorical features. Also the id columns (columns ending in "_id") should be removed from the train, test, and synthetic data.**

In [ ]:
import json
from pprint import pprint

from midst_models.single_table_TabSyn.scripts.eval.eval_density import eval_density
from midst_models.single_table_TabSyn.scripts.eval.eval_quality import eval_quality
from midst_models.single_table_TabSyn.scripts.eval.eval_mle import eval_mle
from midst_models.single_table_TabSyn.scripts.eval.eval_dcr import eval_dcr
from midst_models.single_table_TabSyn.scripts.eval.eval_detection import eval_detection


dataname = "trans"
TRAIN_DATA_PATH = "/projects/aieng/midst_competition/data/clavaddpm/clavaddpm_1/eval_files/trans_no_id.csv"
TEST_DATA_PATH = "/projects/aieng/midst_competition/data/clavaddpm/clavaddpm_1/eval_files/test_no_id.csv"
CLAVADDPM_DATA_PATH = "/projects/aieng/midst_competition/data/clavaddpm/clavaddpm_1/workspace/train_1/trans/_final/trans_synthetic_no_id.csv"
INFO_PATH = "configs/info_files/trans_info.json"

# Density Estimation of Single Column and Pair-wise Correlation

Two metrics are computed in this section: column shapes and column pair trends.
We explain each in the following subsections.

## Single Column Similarity Score

Essentially, each column in a table represents a single feature which accepts various values from a certain type; e.g. numerical, categorical, datetime or boolean. This feature can be described as a random variable where the values listed under the column are its samples. As a result, the density distribution of each column can be computed and compared between the real data and the synthetic data.

<p align="center">
<img src="https://github.com/user-attachments/assets/bf3c6f63-5928-478a-b947-2b52459cbf9f" width="1000"/>
</p>

The better the distributions match each other, the better the quality of synthetic data.

The similarity of the two distriutions can be measured via different metrics for different data types; for example, **KSComplement (Kolmogorov-Smirnov Complement)** is used for numerical features and **TVComplement (Total Variation Distance Complement)** for categorical features. The overall Column Shape Score for the whole table is the average of column shape similarity scores of all columns.

**KST (Kolmogorov-Smirnov Test)**
computes the cumulative distribution function (CDF) of a numerical random variable for real and synthetic data. Then, finds the maximum different between the two CDFs $M$. Finally, the *KSComplement* score is defined as $1 - M$ so that the higher the score, the more similar the distributions.

<p align="center">
<img src="https://github.com/user-attachments/assets/65cac672-e711-4d36-b239-968f09250d78" width="1000"/>
</p>

**TVD (Total Variation Distance)**
computes the frequency of each category's appearance under a certain column and defines it as the said categorie's probability. Then, it computes the sum of differences of probabilities between real and synthetic data as 
$\delta(R, S) = \frac{1}{2} \sum_{\omega \in \Omega}  | R_\omega - S_\omega |$
.

Here, $\omega$ describes all the possible categories in a column, $\Omega$. Meanwhile, $R$ and $S$ refer to the real and synthetic frequencies for those categories. The *TVComplement* returns $1-TVD$ so that a higher score means higher quality.

<p align="center">
<img src="https://github.com/user-attachments/assets/26869056-b1b7-46d0-85e3-668f7a30ff91" width="1000"/>
</p>

## Pair-wise Correlation Score

The correlation between two random variables describes how they vary in relation to each other. The higher the score, the more the trends are alike.

<p align="center">
<img src="https://github.com/user-attachments/assets/1ef5b542-0fdd-4326-bedd-1c93d59fdd6f" width="900"/>
</p>

Here, we use different metrics to compute the correlation between different pairs of data types:

| Column Type | Metric |
| ----------- | ------ |
| numerical & numerical | [correlation similarity](https://docs.sdv.dev/sdmetrics/metrics/metrics-glossary/correlationsimilarity) |
| categorical & categorical | [contingency similarity](https://docs.sdv.dev/sdmetrics/metrics/metrics-glossary/contingencysimilarity) |
| numerical & categorical | discretize the numerical columns into bins, then apply contingency similarity. |

This yields a score between every pair of columns. The **Column Pair Trends** score is the average of all the scores.

In [ ]:
shape, trend = eval_density(CLAVADDPM_DATA_PATH, TRAIN_DATA_PATH, INFO_PATH)
print("Shape:", shape)
print("Trend:", trend)

# $\alpha$-Precision and $\beta$-Recall 

$\alpha$-Preicison and $\beta$-Recall are generalizations of Precision and Recall metrics proposed by [Sajjadi et al.](https://proceedings.neurips.cc/paper/2018/hash/f7696a9b362ac5a51c3dc8f098b73923-Abstract.html) in 2018. These metrics can range between $[0, 1]$ and the closest they are to $1$ the better.


Preicison measures the fidelity or quality of sythetic data. In more clear terms, it computes the proporation of synthetic datapoints that are *close* to real datapoints. 
Recall, on the other hand, measures the diversity of synthetic data; i.e. the extent to which these samples cover the full variability of real samples. More clearly, recall computes the proportion of real datapoints that are *close* to synthetic datapoints.


If we denote the distibution of real datapoints by $P(X)$ and the distribution of sythetic datapoints by $Q(Y)$, precision is the portion of $Q(Y)$ that can be generated by $P(X)$, while recall is the portion of $P(X)$ that can be generated by $Q(Y)$.

To better understand these concepts, let's assume that the real/generated dataponits are samples from an underlying real/generated manifold.
Precision measures the proportation of generated datapoints that fall on the real manifold, while recall measures the proportion of real datapoints that fall on the generated manifold.

<p align="center">
<img src="https://github.com/user-attachments/assets/7b3ace2b-e211-4c64-bf0f-be593513b2fa" width="300"/>
</p>

<p align="center">
<img src="https://github.com/user-attachments/assets/a01e0f96-4a9b-49a9-a7ca-37a484a34fc6" width="300"/>
</p>

The Precision-Recall metrics are very sensitive to outliers since even a few outliers can greatly change the shape of the underlying manifold.
To address this limitation, $\pmb{\alpha}$**-Precision** and $\pmb{\beta}$**-Recall** are defined by assuming that a fraction $1−\alpha$ (or $1−\beta$) of the real (and synthetic) data are “outliers”, and $\alpha$ (or $\beta$) are “typical”. 
$\alpha$-Precision is the fraction of synthetic samples that resemble the “most typical” fraction $\alpha$ of real samples, whereas $\beta$-Recall is the fraction of real samples covered by the most typical fraction $\beta$ of synthetic samples.
The two metrics are evaluated for all $\alpha, \beta \in [0, 1]$, providing entire precision and recall curves instead of single numbers.


To illustrate, consider the below image. Blue and red points are real and generated datapoints, respectively. The large blue and red spheres show the underlying manifold that was estimated from real and generated datapoints.
Good quality generated datapoints should fall within the blue sphere like image *(c)*. They should not lie far from the blue sphere like *(a)*. Moreover, they should not be placed too close (or *copied*) to a real datapoint like *(b)*.
Image *(d)* shows an outlier in the real datapoints which is cut outside of the manifold due to the application on $\alpha$ and $\beta$.
If we used vanilla Precision and Recall, the blue sphere's radius should have increased to include the outlier which would also lead it to include noisy synthetic datapoints like *(a)*.

<p align="center">
<img src="https://github.com/user-attachments/assets/2199f409-20fe-40cb-a7b0-8a4fcc53df3e" width="500"/>
</p>


In [ ]:
alpha_precision_all, beta_recall_all = eval_quality(
    CLAVADDPM_DATA_PATH, TRAIN_DATA_PATH, INFO_PATH
)
print("Alpha precision:", alpha_precision_all)
print("Beta recall:", beta_recall_all)

# Machine Learning Efficiency

This method trains a machine learning model (in our case, an XGBoost model) on the synthetic data and evaluates it on the real data.

In [ ]:
## does a grid search over given params and reports all scores for each best of them
# tabular dataload and tabular transformer look extra
overall_score = eval_mle(CLAVADDPM_DATA_PATH, TEST_DATA_PATH, INFO_PATH)
print("CLAVADDPM - Overall score:")
pprint(overall_score)

As baseline, we also evaluate a similar ML model (i.e. XGBoost) on the real training data.

In [ ]:
overall_score = eval_mle(TRAIN_DATA_PATH, TEST_DATA_PATH, INFO_PATH)
print("BASELINE - Overall score:")
pprint(overall_score)

# Privacy Protection: Distance to Closest Record (DCR)

One of the applications of synthetically generated data is protecting sensitive information while creating similar substitute data that could be used to train machine learning models or published on public platforms.
For this purpose, we must ensure that the synthetic datapoints are far enough from any real datapoints to prevent leaking of real sensitive information.

One metric that is used for this purpose is **Distance to Closest Record (DCR)**.
DCR is the Euclidean distance between a synthetic datapoint and its nearest real datapoint.
DCR equal to zero means that the synthetic datapoint will leak the real information, while higher DCR values mean less risk of privacy leakage.

`eval_dcr` computes the DCR of each synthetic datapoint to real datapoints in two different sets: training and test. Then, it returns the proportion of synthetic datapoints that are closer to the training dataset than the test dataset.
If the size of the training and test datasets are equal, this score should ideally be $0.5$ indicating that the model has not overfit to training data and the synthetic datapoints are not memorized copies of training data.
If the size of the training and test datasets are different, the ideal value for this score is #Train / (#Train + #Test).

In [24]:
# review json file and its contents
with open(INFO_PATH, "r") as file:
    data_info = json.load(file)

In [ ]:
ideal_dcr = data_info["train_num"] / (data_info["train_num"] + data_info["test_num"])

dcr_score = eval_dcr(CLAVADDPM_DATA_PATH, TRAIN_DATA_PATH, TEST_DATA_PATH, INFO_PATH)
print(f"DCR Score, a value closer to {ideal_dcr} is better")
print("Distance to Closest Record:", dcr_score)

# Detection: Classifier Two Sample Tests (C2ST)

This metric evaluates if the synthetic data can be detected from the real data via a machine learning model, hence measuring how difficult it is to distinguish synthetic from real data. A logistic regression model is used in `eval_detection`.

This score is measured through below steps:
1. Create a single, augmented table that has all the rows of real data and all the rows of synthetic data. Add an extra column to keep track of whether each original row is real or synthetic.
2. Split the augmented data to create a training and validation sets.
3. Choose a machine learning model. Train the model on the training split. The model will predict whether each row is real or synthetic (i.e. predict the extra column we created in step #1).
4. Validate the model on the validation set.
5. Repeat steps #2-4 multiple times.
The final score is based on the average ROC-AUC score across all the cross validation splits,

$score = 1 - (max($ <span style="text-decoration:overline">ROC-AUC</span> $, 0.5) \times 2 - 1)$
.

This score can range between $[0, 1]$ with $0$ being the lowest (meaning that the machine learning model can perfectly identify synthetic data apart from the real data), and $1$ being the highest (meaning that the machine learning model cannot identify the synthetic data apart from the real data).



In [ ]:
detection_score = eval_detection(
    CLAVADDPM_DATA_PATH, TRAIN_DATA_PATH, INFO_PATH, dataname, model="clavaddpm"
)
print("CLAVADDPM - Detection score:", detection_score)

In [ ]:
detection_score = eval_detection(
    TRAIN_DATA_PATH, TRAIN_DATA_PATH, INFO_PATH, dataname, model="clavaddpm"
)
print("BASELINE - Detection score:", detection_score)